### Importing Libraries

In [2]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import mediapipe as mp

### Checking our Webcam

In [3]:
cap = cv2.VideoCapture(0) # Accessing our webcam where(0) is the webcam 
while cap.isOpened(): # checking if our accessing our webcam creating a loop
    ret, frame = cap.read() # reading frame from webcam
    cv2.imshow('OpenCV Feed', frame) # show to the user
    if cv2.waitKey(10) & 0xFF == ord('q'): # Break gracefully
        break
cap.release() # release our webcam
cv2.destroyAllWindows() # close down our frame

### Setting up MediaPipe Holestic

In [3]:
mp_holistic = mp.solutions.holistic # Bringing our holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [5]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic: # Set mediapipe model
    while cap.isOpened():
        ret, frame = cap.read()
        
        image,results = mediapipe_detection(frame, holistic) # Make detection
        
        cv2.imshow('OpenCV Feed', frame) 
        if cv2.waitKey(10) & 0xFF == ord('q'): 
            break
    cap.release() 
    cv2.destroyAllWindows() 

In [6]:
#results

In [7]:
#results.face_landmarks

In [8]:
#results.face_landmarks.landmark

In [9]:
#len(results.face_landmarks.landmark)

### Visulalizing landmarks

In [10]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [11]:
#draw_landmarks(frame, results) #drawing landmarks to the current frame

In [12]:
#plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)) #taking the last frame

In [13]:
# cap = cv2.VideoCapture(0)
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#     while cap.isOpened():

#         # Read feed
#         ret, frame = cap.read()
#         image, results = mediapipe_detection(frame, holistic)
#         print(results)
        
#         draw_landmarks(image, results) # Draw landmarks

#         cv2.imshow('OpenCV Feed', image)
#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break
#     cap.release()
#     cv2.destroyAllWindows()

### Editing look of the landmarks

In [14]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [15]:
# cap = cv2.VideoCapture(0)
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#     while cap.isOpened():

#         # Read feed
#         ret, frame = cap.read()
#         image, results = mediapipe_detection(frame, holistic)
#         print(results)
        
#         draw_styled_landmarks(image, results) # Draw landmarks

#         cv2.imshow('OpenCV Feed', image)
#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break
#     cap.release()
#     cv2.destroyAllWindows()

### Extract keypoints value

If we dont have a landmark in the frame we will replace it with a blank array (Zeros) with same shape.

In [16]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)

In [17]:
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)

In [18]:
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)

In [19]:
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [20]:
def extract_keypoints(results): # defining a function for them
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh]) #concatenating so tham we can use them together

### Setting up folder for array collection

In [21]:
DATA_PATH = os.path.join('Training_Test_Data') # Path for exported data, numpy arrays

actions = np.array(['hello', 'thankyou', 'iloveyou', 'sorry', 'please', 'Goodbye']) # Actions that we try to detect

no_sequences = 30 # Thirty videos worth of data

sequence_length = 30 # Videos are going to be 30 frames in length

In [22]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

### Collect Keypoint Values for Training and Testing

In [23]:
# cap = cv2.VideoCapture(0)
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    
#     for action in actions: # Loop through actions
#         for sequence in range(no_sequences): # Loop through sequences aka videos
#             for frame_num in range(sequence_length):   # Loop through video length aka sequence length

#                 ret, frame = cap.read()

#                 image, results = mediapipe_detection(frame, holistic)

#                 draw_styled_landmarks(image, results)
                
#                 if frame_num == 0:   # NEW Apply wait logic
#                     cv2.putText(image, 'STARTING COLLECTION', (120,200), 
#                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
#                     cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
#                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
#                     cv2.imshow('OpenCV Feed', image)
#                     cv2.waitKey(2000)
#                 else: 
#                     cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
#                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
#                     cv2.imshow('OpenCV Feed', image)
                
#                 keypoints = extract_keypoints(results)  # NEW Export keypoints
#                 npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
#                 np.save(npy_path, keypoints)

#                 if cv2.waitKey(10) & 0xFF == ord('q'):
#                     break
                    
#     cap.release()
#     cv2.destroyAllWindows()

### Preprocess Data and Create Labels and Features  

In [24]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [25]:
label_map = {label:num for num, label in enumerate(actions)}

In [26]:
label_map

{'hello': 0,
 'thankyou': 1,
 'iloveyou': 2,
 'sorry': 3,
 'please': 4,
 'Goodbye': 5}

In [27]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [28]:
np.array(sequences).shape

(180, 30, 1662)

In [29]:
np.array(labels).shape

(180,)

In [30]:
X = np.array(sequences)

In [31]:
X.shape

(180, 30, 1662)

In [32]:
y = to_categorical(labels).astype(int)

In [33]:
y.shape

(180, 6)

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

### Build and train LSTM Network

In [35]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [36]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [37]:
[actions.shape[0]]

[6]

In [38]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [39]:
#model.fit(X_train, y_train, epochs=150)

In [40]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 6)                 198       
                                                                 
Total params: 596,774
Trainable params: 596,774
Non-trai

### Make Predictions

In [41]:
res = model.predict(X_test)

In [42]:
actions[np.argmax(res[1])]

'sorry'

In [43]:
actions[np.argmax(y_test[1])]

'please'

### save weights

In [44]:
#model.save('Saved_Model.h5')

In [45]:
model.load_weights('Saved_Model.h5')

### Evaluation

In [46]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [47]:
ypred = model.predict(X_test)

In [48]:
ytrue = np.argmax(y_test, axis=1).tolist()
ypred = np.argmax(ypred, axis=1).tolist()

In [49]:
multilabel_confusion_matrix(ytrue, ypred)

array([[[6, 1],
        [0, 2]],

       [[8, 0],
        [1, 0]],

       [[8, 0],
        [0, 1]],

       [[4, 0],
        [0, 5]]], dtype=int64)

In [50]:
print('Accuracy of the Model is',(accuracy_score(ytrue, ypred))*100)

Accuracy of the Model is 88.88888888888889


### Testing in Real Time

In [51]:
# # 1. New detection variables
# sequence = []
# sentence = []
# threshold = 0.4

# cap = cv2.VideoCapture(0)
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#     while cap.isOpened():

#         ret, frame = cap.read()
#         image, results = mediapipe_detection(frame, holistic)
#         print(results)
        
#         draw_styled_landmarks(image, results)
        
#          # 2. Prediction logic
#         keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        
#         if len(sequence) == 30:
#             res = model.predict(np.expand_dims(sequence, axis=0))[0]
#             print(actions[np.argmax(res)])
            
#             #Viz logic
#             if res[np.argmax(res)] > threshold: 
#                 if len(sentence) > 0: 
#                     if actions[np.argmax(res)] != sentence[-1]:
#                         sentence.append(actions[np.argmax(res)])
#                 else:
#                     sentence.append(actions[np.argmax(res)])

#             if len(sentence) > 5: 
#                 sentence = sentence[-5:]
                
    
#             cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
#             cv2.putText(image, ' '.join(sentence), (3,30), 
#                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)


#         cv2.imshow('OpenCV Feed', image)
#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break
#     cap.release()
#     cv2.destroyAllWindows()

In [52]:
colors = [(245,117,16), (117,245,16), (16,117,245),(245,17,116), (17,245,196), (116,17,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [54]:
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        ret, frame = cap.read()

        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        draw_styled_landmarks(image, results)
        
        keypoints = extract_keypoints(results)

        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

           
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        
        cv2.imshow('OpenCV Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Goodbye
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Goodbye
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Goodbye
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Goodbye
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Goodbye
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Goodbye
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Goodbye
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Goodbye
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Goodbye
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Goodbye
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Goodbye
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Goodbye
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Goodbye
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Goodbye
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Goodbye
<class 'mediapipe.python.

please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution

<class 'mediapipe.python.solution_base.SolutionOutputs'>
thankyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thankyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thankyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thankyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thankyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thankyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thankyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thankyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thankyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thankyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thankyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thankyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thankyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sorry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
sorry
<class 'mediapip